In [62]:
import sys
sys.path.append("../") # go to parent dir

from pprint import pprint
from pathlib import Path
import pyjson5
from dcf.utils import relative_error, average_dict_val
from dcf.ddm import make_diameter_calculator, make_diffusion_coefficient_calculator

project_path = Path("..").resolve()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
ddm_data_path = project_path / "data" / "DDM" / "ddm_results.json5"
particle_data_path = project_path / "data" / "particle_data.json5"

with open(ddm_data_path) as f:
    ddm_data = pyjson5.load(f)

with open(particle_data_path) as f:
    particle_data = pyjson5.load(f)


In [64]:
# Normalise data
for exp_name, exp in ddm_data["experiments"].items():
    exp |= ddm_data["setups"][exp["setup"]]
    exp |= particle_data.get(exp["particle"], {})


In [66]:
# Compute data
temperature = 273 + 21
viscosity = 0.9775e-3

diffusion_coefficient_calculator = make_diffusion_coefficient_calculator(viscosity=viscosity, temperature=temperature)

for exp_name, exp in ddm_data["experiments"].items():
    diameter_calculator = make_diameter_calculator(viscosity=viscosity, temperature=temperature, pix2mum=exp["pix2mum"], framerate=exp["framerate"])

    exp["exp_diameter2"] = diameter_calculator(exp["exp_diffusion_coeff"])

    if "known_diameter" in exp:
        exp["theoretical_diffusion_coeff"] = diffusion_coefficient_calculator(exp["known_diameter"])
        exp["error"] = relative_error(exp["known_diameter"], exp["exp_diameter2"])

pprint(ddm_data)


{'particle': 'M', 'exp_diffusion_coeff': 1.21, 'exp_diameter': 3.6392e-07, 'concentration': 0.1, 'setup': '20x_1', 'objective': '20x', 'pix2mum': 0.3008, 'framerate': 100.0}


KeyError: 'px2mum'